<a href="https://colab.research.google.com/github/elbonks/Aplicaci-n-IA-que-distingue-perros-y-gatos/blob/main/el_bueno_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Del modelo a la aplicación
En este sprint vamos a construir una aplicación real de Machine Learning en Python para resolver un problema de clasificación de imágenes "Perro/Gato".

En esta primera aproximación **low-code**, usaremos el modelo entrenado en Teachable Machine y construiremos la aplicación sobre Google Colab. Nos centraremos en la inferencia haciendo uso de **TensorFlow/Keras** para cargar el modelo y hacer predicciones.

## Cargar el modelo

Previamente a exportar el modelo entrenado en Teachable Machine a Python, debemos tener claro algunos conceptos clave. Las imágenes utilizadas para entrenar cada una de las categorías puedes descargarlas de ![Cats and Dogs light](https://zenodo.org/records/5226945).

El repositorio contiene imágenes (500 gatos y 500 perros) y 400 imágenes para el conjunto de prueba (200 cada uno).

Una vez entrenado el modelo, lo exportamos en formato **Keras**.

👀: Google Colab ha actualizado la librería Keras a la versión 3.0, mientras que Teachable Machine exporta los modelos en formato Keras 2.x. Por tanto, debemos indicar el siguiente parámetro para garantizar la compatibilidad.

In [12]:
import os
# 1. Activamos el modo legacy (Keras 2)
os.environ['TF_USE_LEGACY_KERAS'] = '1'

El parámetro `compile=False` se usa para evitar que Keras intente compilar el modelo al cargarlo, lo cual no es necesario para la inferencia.

In [13]:
#from tf_keras.models import load_model
from tensorflow import keras
try:
    mi_modelo = keras.models.load_model("/content/drive/MyDrive/keras_model.h5", compile=False)
    print("✅ ¡Éxito! Modelo .h5 cargado con el paquete tf-keras.")
    mi_modelo.summary()
except Exception as e:
    print(f"❌ Error al cargar el modelo: {e}")

✅ ¡Éxito! Modelo .h5 cargado con el paquete tf-keras.
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_9 (Sequential)   (None, 1280)              410208    
                                                                 
 sequential_11 (Sequential)  (None, 2)                 128300    
                                                                 
Total params: 538508 (2.05 MB)
Trainable params: 524428 (2.00 MB)
Non-trainable params: 14080 (55.00 KB)
_________________________________________________________________


## Carga de imágenes

En esta primera aproximación, se cargará la imagen que quieras clasificar a Google Colab.

Para cargar imágenes en Python, usaremos la librería PIL (Python Imaging Library), en concreto la clase Image e ImageOps ('from PIL import Image, ImageOps').

In [14]:
from PIL import Image, ImageOps

imagen= Image.open("/content/drive/MyDrive/cat vs dog/cars/cat.0.jpg").convert("RGB")

La imagen DEBE transformarse para ser idéntica a las de entrenamiento.

Esto incluye:

* Cambiar tamaño de la imagen para que tenga al menos 224x224 píxeles y luego se recorta desde el centro.
* Normalizar los valores de los píxeles (ej. de [0, 255] a [-1, 1]). Para ello deberemos utilizar la librería Numpy (`import numpy as np`)
* Por último. para desplazar el valor entre -1 y +1, le restaremos -1 a cada valor de la celda.


In [15]:
import numpy as np
size = (224, 224)
imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)

imagen_array = np.asarray(imagen)

normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

Las IAs no procesan imágenes "una a una", sino en lotes. Aunque solo tengamos una foto, debemos meterla en un array de 1 hueco. Ese hueco, a su vez, debe contener la imagen pre-procesada.

Para clasificar la imagen preprocesada, usamos el método predict() del modelo cargado en la Fase 1 ('mi_modelo').

predict está diseñado para procesar conjuntos de datos a la vez. Por eso, necesita recibir una lista (un lote), aunque esa lista solo tenga un elemento.

In [16]:
nombres_limpios = ['gato', 'perro']

etiquetas_finales = {
    'gato': 'Gato (Cat)',
    'perro': 'Perro (Dog)'
}

In [17]:
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Load the image into the array
lote_imagenes[0] = normalizada_imagen_array

# Predicts the model
resultados = mi_modelo.predict(lote_imagenes)
print(resultados)

indice = np.argmax(resultados[0])
print("Índice predicho:", indice)

# Obtener el nombre limpio de la clase usando el índice predicho
etiqueta_limpia_predicha = nombres_limpios[indice]

# Obtener la etiqueta final deseada del diccionario de mapeo
clase_final_para_mostrar = etiquetas_finales.get(etiqueta_limpia_predicha, etiqueta_limpia_predicha) # Fallback si no se encuentra

probabilidad = resultados[0][indice]

# Print prediction and confidence score
print("Clase:", clase_final_para_mostrar)
print("Probabilidad:", probabilidad)

1/1 [==============================] - 1s 1s/step
[[1.0000000e+00 1.7959528e-10]]
Índice predicho: 0
Clase: Gato (Cat)
Probabilidad: 1.0


# Test del programa
ahora usaremos nuestras imágenes de test para entrenar a nuestro moledo y ahora ver como de bien funciona

In [ ]:
def predecir_imagen(ruta_imagen):
    # 1. Carga de la imagen
    imagen = Image.open(ruta_imagen).convert("RGB")

    # 2. Preprocesamiento de la imagen
    size = (224, 224)
    imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
    imagen_array = np.asarray(imagen)
    normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

    # 3. Creación del lote de imágenes
    lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    lote_imagenes[0] = normalizada_imagen_array

    # 4. Predicción del modelo
    resultados = mi_modelo.predict(lote_imagenes)
    indice = np.argmax(resultados[0])

    # 5. Obtener etiqueta y probabilidad
    etiqueta_limpia_predicha = nombres_limpios[indice]
    probabilidad = resultados[0][indice]

    return etiqueta_limpia_predicha, probabilidad

lista_archivos = os.listdir("/content/drive/MyDrive/cat vs dog/test")
print(lista_archivos)
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
# predicciones incorrectas contendrá una lista con los nombres de las imágenes mal clasificadas
predicciones_incorrectas = []
for nombre_archivo in lista_archivos:
    if "cat" in nombre_archivo:
        etiqueta_esperada = "gato"
    elif "dog" in nombre_archivo:
        etiqueta_esperada = "perro"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/cat vs dog/test", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función que
    # implemente las fases de inferencia
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)

['dog.10346.jpg', 'dog.10345.jpg', 'dog.10326.jpg', 'dog.10360.jpg', 'dog.10325.jpg', 'dog.10337.jpg', 'dog.10350.jpg', 'dog.10376.jpg', 'dog.10323.jpg', 'dog.10330.jpg', 'dog.10342.jpg', 'dog.10344.jpg', 'dog.10372.jpg', 'dog.1035.jpg', 'dog.10356.jpg', 'dog.10365.jpg', 'dog.10343.jpg', 'dog.10324.jpg', 'dog.10327.jpg', 'dog.10369.jpg', 'dog.10349.jpg', 'dog.10370.jpg', 'dog.10329.jpg', 'dog.10332.jpg', 'dog.1034.jpg', 'dog.10339.jpg', 'dog.10366.jpg', 'dog.10351.jpg', 'dog.10358.jpg', 'dog.10362.jpg', 'dog.1036.jpg', 'dog.10341.jpg', 'dog.10368.jpg', 'dog.10348.jpg', 'dog.10335.jpg', 'dog.10352.jpg', 'dog.10364.jpg', 'dog.10353.jpg', 'dog.1037.jpg', 'dog.10367.jpg', 'dog.10347.jpg', 'dog.10328.jpg', 'dog.10357.jpg', 'dog.1033.jpg', 'dog.10361.jpg', 'dog.10331.jpg', 'dog.10355.jpg', 'dog.10359.jpg', 'dog.10333.jpg', 'dog.10340.jpg', 'dog.10377.jpg', 'dog.10354.jpg', 'dog.10336.jpg', 'dog.10375.jpg', 'dog.10428.jpg', 'dog.1038.jpg', 'dog.10427.jpg', 'dog.10417.jpg', 'dog.10416.jpg', 'd

# Resultados
Después de que el código anterior ha procesado todas las imágenes y ha guardado la información de aciertos y errores, este bloque se encarga de presentarnos un resumen claro de los resultados.

In [19]:
precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos /= aciertos if aciertos > 0 else 1
informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Predicción: {error['prediccion']}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)


Informe de Evaluación del Modelo

Total de predicciones: 1
Aciertos: 0
Precisión: 0.00%
Probabilidad media de aciertos: 0.00%

Predicciones incorrectas:

